In [ ]:
import os, io
import json
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pyarrow.parquet as pq
import pandas as pd
import glob
from torchvision import transforms

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

### Data Loading Class for GSM8K

In [ ]:
class GSM8KDataset(Dataset):
    def __init__(self, split):
        file_path = os.path.join(os.path.dirname(__file__), f"{split}.jsonl")
        print(f"Loading GSM8K dataset from {file_path}...")
        with open(file_path, 'r') as f:
            self.data = [json.loads(line) for line in f]
        print(f"Loaded {len(self.data)} examples.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "question": item["question"],
            "answer": item["answer"]
        }

def get_gsm8k_dataloader(batch_size=32, split="train"):
    dataset = GSM8KDataset(split)
    return DataLoader(dataset, batch_size=batch_size, shuffle=(split == "train"))

### Data Loading Class for MMEDataset

In [ ]:
class MMEDataset(Dataset):
    def __init__(self, root_dir, split="test"):
        self.root_dir = root_dir
        self.split = split
        self.data = self.load_data()
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  
            transforms.ToTensor(),
        ])

    def load_data(self):
        data = []
        file_pattern = os.path.join(self.root_dir, f"{self.split}-*-of-00004-*.parquet")
        parquet_files = sorted(glob.glob(file_pattern))
        
        if not parquet_files:
            raise FileNotFoundError(f"No parquet files found matching pattern: {file_pattern}")

        for file_path in parquet_files:
            print(f"Loading data from {file_path}")
            if not os.path.exists(file_path):
                raise FileNotFoundError(f"Data file not found: {file_path}")
            df = pq.read_table(file_path).to_pandas()
            data.append(df)
        
        data = pd.concat(data, ignore_index=True)
        print(f"Loaded {len(data)} examples.")
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        
        image_bytes = item['image']['bytes']
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        image = self.transform(image)
        
        return {
            "image": image,
            "question": item["question"],
            "answer": item["answer"],
            "question_id": item["question_id"],
            "category": item["category"]
        }

def custom_collate(batch):
    images = torch.stack([item['image'] for item in batch])
    questions = [item['question'] for item in batch]
    answers = [item['answer'] for item in batch]
    question_ids = [item['question_id'] for item in batch]
    categories = [item['category'] for item in batch]

    return {
        'image': images,
        'question': questions,
        'answer': answers,
        'question_id': question_ids,
        'category': categories
    }

def get_mme_dataloader(root_dir, batch_size=1, split="test"):
    dataset = MMEDataset(root_dir, split)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate)

### Qwen2-1.5B Inference

In [ ]:
import os, json
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from data_loaders import get_gsm8k_dataloader
import logging
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

logging.getLogger("transformers").setLevel(logging.ERROR)

def load_qwen_model():
#    model_path = os.path.join(os.path.dirname(__file__), "Qwen1.5-1.8B")
    model_path = "/root/autodl-tmp/Qwen1.5-1.8B"
    print(f"Attempting to load model from local path: {model_path}")
    
    try:
#        print("Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
#        print("Tokenizer loaded successfully.")
        
#        print("Loading model... This may take a while.")
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).to(device)
#        print(f"Model loaded successfully and moved to {device}.")
    except Exception as e:
        print(f"Error loading model or tokenizer: {e}")
        raise
    
    return model, tokenizer, device

def qwen_inference():
    model, tokenizer, device = load_qwen_model()
    dataloader = get_gsm8k_dataloader(batch_size=1, split="test")
    total_samples = len(dataloader)

    results = []
    process_single_sample = False
    
    for i, batch in enumerate(dataloader):        
        question = batch["question"][0]
        print(f"  Question: {question[:50]}...")
        
        prompt = f"Question: {question}\nAnswer:"
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        
        try:
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=50, max_time=30,
                                         num_return_sequences=1, do_sample=True, 
                                         temperature=0.7, top_p=0.95,
                                         pad_token_id=tokenizer.eos_token_id)
            
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            answer = generated_text.split("Answer:")[-1].strip()
            print(f"  Answer generated: {answer[:50]}...")
        except Exception as e:
            print(f"  Error generating answer: {str(e)}")
            answer = "Error: Failed to generate answer"
        
        results.append({
            "question": question,
            "generated_answer": answer,
            "ground_truth": batch["answer"][0]
        })
        
        torch.cuda.empty_cache()  
        
        if process_single_sample:
            break 
    
    # Monitor GPU memory usage
    print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"GPU memory reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    
    return results

if __name__ == "__main__":
    results = qwen_inference()
    print(f"Processed {len(results)} questions from GSM8K dataset")

### MiniGPT-4 Inference

In [ ]:
import os
import sys
import torch
from torch.utils.data import Dataset, DataLoader
from data_loaders import MMEDataset, get_mme_dataloader
from PIL import Image
import pandas as pd
import pyarrow.parquet as pq
import glob
from tqdm import tqdm
import csv

sys.path.append(os.path.expanduser("~/MiniGPT-4"))

from minigpt4.common.config import Config
from minigpt4.common.registry import registry
from minigpt4.conversation.conversation import Chat, CONV_VISION_Vicuna0, CONV_VISION_LLama2, StoppingCriteriaSub

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def run_inference(chat, dataloader):
    results = []
    for batch in tqdm(dataloader, desc="Running inference"):
        image = batch["image"][0]
        question = batch["question"][0]
        ground_truth = batch["answer"][0]

        chat_state = CONV_VISION.copy()
        chat_state.system = "You are a helpful AI assistant."
        chat.upload_img(image, chat_state, use_cache=False)
        answer, _ = chat.answer(question, chat_state)

        results.append({
            "question": question,
            "ground_truth": ground_truth,
            "prediction": answer
        })

    return results

def save_results(results, output_path):
    with open(output_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=["question", "ground_truth", "prediction"])
        writer.writeheader()
        writer.writerows(results)

def main():
    cfg = Config(parse_args=False)
    cfg.model_config = "/MiniGPT-4/eval_configs/minigpt4_eval.yaml"
    cfg.options = ["model.device='cuda'"]
    cfg.ckpt = "prerained_minigpt4_7b.pth"  

    model_config = cfg.model_config
    model_cls = registry.get_model_class(model_config.arch)
    model = model_cls.from_config(model_config).to('cuda')

    model.load_checkpoint(cfg.ckpt)

    chat = Chat(model)

    root_path = "."
    dataloader = get_mme_dataloader(root_path, batch_size=1)

    results = run_inference(chat, dataloader)

    output_path = "minigpt4_results.csv"
    save_results(results, output_path)
    print(f"Results saved to {output_path}")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
from qwen_inference import qwen_inference
#
import time, argparse
import sys
import os, json, io
sys.path.append(os.path.abspath('/root/MiniGPT-4'))
import subprocess
from data_loaders import MMEDataset

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

def create_result_table(results, model_name):
    df = pd.DataFrame(results)
    df["model"] = model_name
    return df

def parse_args():
    parser = argparse.ArgumentParser(description="Run inference on MME dataset using MiniGPT-4")
    parser.add_argument("--cfg_path", default="model/MiniGPT-4/eval_configs/minigpt4_eval.yaml", type=str,
                        help="Path to MiniGPT-4 config file")
    parser.add_argument("--gpu_id", type=int, default=0, help="GPU ID to use")
    parser.add_argument("--data_path", default="/root", type=str, help="Path to MME dataset")
    parser.add_argument("--output_path", default="/root/minigpt4_results.csv", type=str, 
                        help="Path to save results (CSV)")
    return parser.parse_args()

def main():
    print("\nRunning Qwen2-1.5b inference on GSM8K dataset...")
    qwen_results = qwen_inference()
    qwen_df = create_result_table(qwen_results, "Qwen2-1.5b")
    qwen_df.to_csv("qwen_results.csv", index=False)
    print("Qwen2-1.5b results saved to qwen_results.csv")

    print("\nRunning MiniGPT-4 inference on MME dataset...")
    # minigpt4_start = time.time()
    # args = parse_args()

    # # Load MME dataset
    # dataset = MMEDataset(args.data_path)

    # # Initialize MiniGPT-4 inference
    # minigpt4_model = MiniGPT4Inference(args.cfg_path, args.gpu_id)
    # results = minigpt4_model.run_inference(dataset)

    # # Convert results to DataFrame
    # results_df = pd.DataFrame(results)

    # # Save results to CSV
    # results_df.to_csv(args.output_path, index=False)

    # print(f"Inference completed. Results saved to {args.output_path}")

if __name__ == "__main__":
    main()